In [2]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 6.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 8.9 MB/s 
     |████████████████████████████████| 70 kB 11.8 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.1-cp37-cp37m-linux_x86_64.whl size=629484 sha256=3685bdd3d319c99f4c94cd6f1c28498e1b3d1b391436be72f27721599941b2ed
  Stored in directory: /root/.cache/pip/wheels/17/53/c9/caa05618e686df51f017d8a9923f38d915ce31df67ab6628e6
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65034 sha256=995bfa34aa7168c8dd1fe963c74ad58f8ce5ed2a866cd976be68fad30d53978f
  Stored in directory: /root/.cache/pip/wheels/37/5e/9e/76d7430e116b7cab0016fbabb26b896daae1946a3f7dea9915
Successfully built pycuda pytools


In [3]:
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np
import math

## MEC(minimum energy control)

In [4]:
class MinimumEnergyControl:
    def __init__(self, x_des, x_0, step=50, dt=0.05, damping=False):

        ## gravity, criterion: moon
        gravity = -1.62     # N/kg

        ## no drag or something disturb movement
        if not damping:
            ## A
            self.state_transition_matrix = \
                np.array([[ 1, 0, 0,dt, 0, 0],
                          [ 0, 1, 0, 0,dt, 0],
                          [ 0, 0, 1, 0, 0,dt],
                          [ 0, 0, 0, 1, 0, 0],
                          [ 0, 0, 0, 0, 1, 0],
                          [ 0, 0, 0, 0, 0, 1]])

            ## B
            input_matrix = \
                np.array([[dt*dt/2,      0,      0],
                          [      0,dt*dt/2,      0],
                          [      0,      0,dt*dt/2],
                          [     dt,      0,      0],
                          [      0,     dt,      0],
                          [      0,      0,     dt]])
            
            self.input_matrix = \
                gpuarray.to_gpu(np.float32(np.array([0.5*dt*dt, dt])))

            ## g
            gravity_matrix = \
                np.array([[              0],
                          [              0],
                          [gravity*dt*dt/2],
                          [              0],
                          [              0],
                          [     gravity*dt]])
                
            self.gravity_matrix = \
                gpuarray.to_gpu(np.float32(np.array([0.5*gravity*dt*dt, gravity*dt])))

        ## drag or something exist...
        else:
            pass

        ## desired state: x_des
        self.x_des = gpuarray.to_gpu(np.float32(x_des))

        ## initial state: x_0
        self.x_0 = gpuarray.to_gpu(np.float32(x_0))

        self.dt = np.float32(dt)

        self.step = step

        ## weight
        self.rho = 1/3

        self.rho_matrix = \
            gpuarray.to_gpu(np.float32(math.sqrt(self.rho) * np.identity(3*self.step)))
 
        ## solution!!!
        self.u = gpuarray.to_gpu(np.float32(np.zeros((3*self.step,1))))

        ## G, gram_G, Q
        self.G = gpuarray.to_gpu(np.float32(np.zeros((3*self.step,6)).reshape(6*3*self.step)))
        self.gram_G = gpuarray.to_gpu(np.float32(np.zeros((3*self.step,3*self.step)).reshape(3*3*self.step*self.step)))
        self.Q = gpuarray.to_gpu(np.float32(np.zeros((6,1))))
        self.C = gpuarray.to_gpu(np.float32(np.zeros((6,1))))
        self.G_C = gpuarray.to_gpu(np.float32(np.zeros((150,1))))
        self.gradient = gpuarray.to_gpu(np.float32(np.zeros((150,1))))

        ## TPB: thread_per_block, BPG: block_per_grid
        self.TPB, self.iteration = self.optimal_size(3*self.step) 

    def run(self):
        self.get_gradient(self.gram_G,
                              self.u,
                              self.G_C,          
                              self.iteration,
                              self.gradient,
                              block=(self.TPB,1,1),
                              grid=(3*self.step,1,1))

    def optimal_size(self, n):
        thread_per_block = int(math.sqrt(n/2))

        iteration = int(n / thread_per_block) + 1

        return thread_per_block, np.int32(iteration)

    def define_object_function_at_kernel(self):
        self.ker_function()

        self.get_G_matrix(self.input_matrix, self.dt, self.G, block=(6,1,1), grid=(self.step,1,1))
        self.get_Q_matrix(self.gravity_matrix, self.dt, self.Q, block=(self.step,1,1), grid=(2,1,1))
        self.get_G_gram_matrix(self.G, self.rho_matrix, self.gram_G, block=(3,1,1), grid=(self.step,self.step,1))
        self.get_G_C_matrix(self.G, self.x_des, self.x_0, self.Q, self.C, self.G_C, block=(3,1,1), grid=(self.step,1,1))

    def ker_function(self):
        ## We'll gonna do 150 x 150 @ 150 x 1
        ## block=(thread_per_block,1,1), grid=(3*self.step,1,1)
        get_gradient_ker_function = \
        """
        #define tx (threadIdx.x)
        #define bx (blockIdx.x)
        #define bs (blockDim.x)
        #define gs (gridDim.x)

        __global__ void get_gradient(float* matrix, float* vector1, float* vector2, int iteration, float* result) {

            __shared__ float result_jerk[1000];

            result_jerk[tx] = 0.0;

            for (int i = 0; i < iteration; i++) {
                int index1 = i + tx * iteration;
                int index2 = index1 + bx * 150;

                if (index1 < gs) {
                    result_jerk[tx] += matrix[index2] * vector1[index1];
                }
                else {
                    result_jerk[1000-tx] = 0;
                }
            }

            __syncthreads();

            if (tx == 0) {
                for (int j = 0; j < bs; j++) {
                    result[bx] += result_jerk[j];
                }

                result[bx] -= vector2[bx];
            }
            else {
                result_jerk[1000-tx] = 0;
            }

            __syncthreads();
        }
        """
        get_gradient_ker = SourceModule(get_gradient_ker_function)

        ## block=(6,1,1), grid=(self.step,1,1)
        get_G_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)
        #define step (gridDim.x)

        __global__ void get_G_matrix(float* input_matrix, float dt, float* G) {
            // 6: DOF, 18: DOF*axis
            int index = tx + (tx%3) * 6 + bx * 18;

            if (tx < 3) {
                float value;
                value = input_matrix[0] + (step - bx - 1) * input_matrix[1];

                G[index] = value;
            }
            else {
                G[index] = dt;
            }

            __syncthreads();
        }
        """
        get_G_matrix_ker = SourceModule(get_G_matrix_ker_function)

        ## block=(self.step,1,1), grid=(2,1,1)
        get_Q_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)
        #define step (blockDim.x)

        __global__ void get_Q_matrix(float* gravity, float dt, float* Q) {
            
            __shared__ float value[50];
            
            if (bx == 0) {
                value[tx] = gravity[0] + (tx * dt) * gravity[1];
            }
            else {
                value[tx] = gravity[1];
            }

            __syncthreads();

            if (bx == 0) {
                if(tx == 0) {
                    for (int i = 0; i < step; i++) {
                        Q[2] += value[i];
                    }
                }
            }
            else {
                if(tx == 0) {
                    for (int i = 0; i < step; i++) {
                        Q[5] += value[i];
                    }
                }
            }

            __syncthreads();
        }
        """
        get_Q_matrix_ker = SourceModule(get_Q_matrix_ker_function)

        ## block=(3,1,1), grid=(self.step,self.step,1)
        get_G_gram_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define by (blockIdx.y)
        #define tx (threadIdx.x)
        #define step (gridDim.x)

        __global__ void get_G_gram_matrix(float* G, float* rho_matrix, float* gram_G) {
            // 9: axis, 151: axis*step+1, 450: axis*axis*step
            int index1 = tx * 151 + bx * 3 + by * 450;

            // 7: DOF+1, 18: DOF*axis
            int index2 = tx * 7 + bx * 18;
                
            float value;
            value = G[index2] * G[index2] + G[index2+3] * G[index2+3];

            gram_G[index1] = value; 

            __syncthreads();

            gram_G[index1] += rho_matrix[index1]*rho_matrix[index1];

            __syncthreads();
        }
        """
        get_G_gram_matrix_ker = SourceModule(get_G_gram_matrix_ker_function)

        ## block=(3,1,1), grid=(self.step,1,1)
        get_G_C_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)

        __global__ void get_G_C_matrix(float* G, float* x_des, float* x_0, float* Q, float* C, float* G_C) {
            // C first in each block
            __shared__ float C_jerk[6];

            C_jerk[tx] = x_des[tx] - Q[tx] - x_0[tx];
            C_jerk[tx+3] = x_des[tx+3] - Q[tx+3] - x_0[tx+3];

            __syncthreads();

            C[tx] = C_jerk[tx];
            C[tx+3] = C_jerk[tx+3];

            __syncthreads();

            // G_C Next
            int index1 = tx * 7 + bx * 18;
            int index2 = tx + bx * 3;

            float value;
            value = G[index1] * C_jerk[tx] + G[index1+3] * C_jerk[tx+3];

            __syncthreads();

            G_C[index2] = value;

            __syncthreads();
        }
        """
        get_G_C_matrix_ker = SourceModule(get_G_C_matrix_ker_function)

        self.get_G_matrix = get_G_matrix_ker.get_function("get_G_matrix")
        self.get_Q_matrix = get_Q_matrix_ker.get_function("get_Q_matrix")
        self.get_G_gram_matrix = get_G_gram_matrix_ker.get_function("get_G_gram_matrix")
        self.get_G_C_matrix = get_G_C_matrix_ker.get_function("get_G_C_matrix")
        self.get_gradient = get_gradient_ker.get_function("get_gradient")

In [5]:
class OptimizerForGuidance:
    def __init__(self, length, learning_rate):
        self.length = length
        self.learning_rate = np.float32(learning_rate)
        self.kernel_function()

    def run(self, theta, gradient):
        ## theta, gradient: gpuarray type variable
        self.basic_optimizer(theta,
                             gradient,
                             self.learning_rate,
                             block=(self.length,1,1),
                             grid=(1,1,1))

    def kernel_function(self):
        ## block=(length,1,1), grid=(1,1,1)
        basic_optimizer_ker_function = \
        """
        #define x (threadIdx.x)

        __global__ void basic_optimizer(float* theta, float* gradient, float learning_rate) {
            theta[x] -= gradient[x] * learning_rate;

            __syncthreads();
        }
        """
        basic_optimizer_ker = SourceModule(basic_optimizer_ker_function)

        self.basic_optimizer = basic_optimizer_ker.get_function("basic_optimizer")

In [6]:
class MinimumEnergyControlSolver:
    def __init__(self, x_des, x_0):
        ## initialize MEC
        self.MEC = MinimumEnergyControl(x_des, x_0)
        self.MEC.define_object_function_at_kernel()

        ## initialize optimizer
        learning_rate = 1e-4
        self.optimizer = OptimizerForGuidance(3*self.MEC.step, learning_rate)

    def solve(self):
        for i in range(100):
            ## get_gradient
            self.MEC.run()

            ## optimize
            self.optimizer.run(self.MEC.u, self.MEC.gradient)

## Test

In [7]:
## destination
x_des = np.array([0,0,0,0,0,0])

## initial point
x_0 = np.array([100,0,-1500,-10,0,80])

MECS = MinimumEnergyControlSolver(x_des, x_0)

In [8]:
MECS.solve()

MECS.MEC.u[:12]

array([[-32.41489 ],
       [  0.      ],
       [487.9284  ],
       [-29.959936],
       [  0.      ],
       [450.98013 ],
       [-27.504957],
       [  0.      ],
       [414.03116 ],
       [-25.050003],
       [  0.      ],
       [377.08206 ]], dtype=float32)

Bravo!

## Constraint

In [9]:
class ConstraintsForInput:
    def __init__(self, problem, upper_boundary, downer_boundary):
        ## ex> MEC(minimum energy control)
        self.problem = problem

        self.upper_boundary = np.float32(upper_boundary)
        self.downer_boundary = np.float32(downer_boundary)

        self.kernel_function()

    def projection(self):
        self.project_function(self.problem.u,
                              self.upper_boundary,
                              self.downer_boundary,
                              block=(3,1,1),
                              grid=(self.problem.step,1,1))

    def kernel_function(self):
        ## block=(3,1,1), grid=(problem.step,1,1)
        projection_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)

        __device__ float square_root(float value) {
            float s = 0;
            float t = 0;

            s = value / 2;

            for (;s != t;) {
                t = s;
                s = ((value/t) + t) / 2;
            }

            return s;
        }

        __device__ float get_norm(float x, float y, float z) {
            float value;
            float norm;

            value = x * x + y * y + z * z;
            norm = square_root(value);

            return norm;    
        }

        __global__ void projection(float* theta, float upper_boundary, float downer_boundary) {
            __shared__ float u[3];
            __shared__ float norm[1];
            float value;

            int index = tx + bx * 3;

            u[tx] = theta[index];

            __syncthreads();

            if (tx == 0) {
                norm[0] = get_norm(u[0], u[1], u[2]);
            } 

            __syncthreads();

            if ((norm[0] > downer_boundary) && (norm[0] < upper_boundary)) {
                value = u[tx];
            }
            else {
                value = u[tx] * upper_boundary / norm[0];
            }

            __syncthreads();

            theta[index] = value;
        }
        """
        projection_ker = SourceModule(projection_ker_function)

        self.project_function = projection_ker.get_function("projection")

## Redefine Solver

In [51]:
class MinimumEnergyControlSolver:
    def __init__(self, x_des, x_0, upper_boundary, downer_boundary, max_iteration=100):
        ## max_iteration
        self.max_iteration = max_iteration

        ## initialize MEC(minimum energy control)
        self.MEC = MinimumEnergyControl(x_des, x_0)
        self.MEC.define_object_function_at_kernel()

        ## initialize optimizer
        learning_rate = 1e-4

        self.optimizer = OptimizerForGuidance(3*self.MEC.step, learning_rate)

        ## constraint
        self.upper_boundary = upper_boundary
        self.downer_boundary = downer_boundary

        self.constraint = ConstraintsForInput(self.MEC, self.upper_boundary, self.downer_boundary)

        ## evaluate
        self.error_vector = gpuarray.to_gpu(np.float32(np.zeros((3*self.MEC.step+6,1))))
        self.error = gpuarray.to_gpu(np.float32(np.zeros((1,self.max_iteration)))) 

        ## TPB = 5, iteration = 10
        self.TPB, self.iteration = self.MEC.optimal_size(self.MEC.step)

        self.kernel_function()

    def solve(self):
        for i in range(100):
            ## get_gradient
            self.MEC.run()

            ## optimize
            self.optimizer.run(self.MEC.u, self.MEC.gradient)

            ## constraint
            self.constraint.projection()

            ## evaluate
            self.evaluate(i)


    def evaluate(self, current_iter):
        
        ## evaluate learning
        self.get_error_vector(self.MEC.G,
                              self.MEC.rho_matrix,
                              self.MEC.u,
                              self.MEC.C,
                              self.iteration, 
                              self.error_vector, 
                              block=(self.TPB,1,1),
                              grid=(156,1,1))
        
        self.get_error(self.error_vector,
                       self.error,
                       np.int32(current_iter),
                       block=(156,1,1),
                       grid=(1,1,1))
        
    def kernel_function(self):
        ## We'll gonna do 156 x 150 @ 150 x 1
        ## block=(TPB,1,1), grid=(156,1,1)
        get_error_vector_ker_function = \
        """
        #define tx (threadIdx.x)
        #define bx (blockIdx.x)
        #define bs (blockDim.x)

        __global__ void get_error_vector(float* G, float* rho_matrix, float* u, float* C, int iteration, float* error_vector) {

            if (bx < 6) {
                
                __shared__ float value[100];

                value[tx] = 0.0;

                __syncthreads();

                for (int i = 0; i < iteration; i++) {
                    int index1 = bx % 3;
                    int index2 = i * 5 + tx % 5;
                    int index3 = index1 + index2*3;

                    // 7: DOF+1, 90: 5*DOF*axis
                    int index4 = bx + index1*6 + index2*18;

                    value[tx] += G[index4] * u[index3];
                }

                __syncthreads();
                if (tx == 0) {
                    // initialize
                    error_vector[bx] = 0.0;

                    value[50] = 0.0;
                    for (int j = 0; j < bs; j++) {
                        value[50] += value[j];
                    }
                 
                    error_vector[bx] = value[50] - C[bx];
                }
                __syncthreads();
            }
            else {
                if (tx == 0) {
                    // initialize
                    error_vector[bx] = 0.0;

                    int index1 = bx - 6;
                    int index2 = index1 * 151;

                    error_vector[bx] = rho_matrix[index2] * u[index1];
                }

                __syncthreads();
            }
        }
        """

        ## block=(156,1,1), grid=(1,1,1)
        get_error_ker_function = \
        """
        #define tx (threadIdx.x)
        #define bs (blockDim.x)

        __device__ float square_root(float value) {
            float s = 0;
            float t = 0;

            s = value / 2;

            for (;s != t;) {
                t = s;
                s = ((value/t) + t) / 2;
            }

            return s;
        }

        __device__ float get_norm(float* vector, int length) {
            float value = 0.0;
            float norm;

            for (int i = 0; i < length; i++) {
                value += vector[i] * vector[i];
            } 

            norm = square_root(value);

            return norm;    
        }


        __global__ void get_error(float* error_vector, float* error, int current_iter) {

            __shared__ float value[1000];

            value[tx] = error_vector[tx];

            __syncthreads();

            if (tx == 0) {
                int length = bs;

                error[current_iter] = get_norm(value, length);
            }
            else {

                value[1000-tx] = 0.0;
            }
            
            __syncthreads();
        }
        """
        get_error_vector_ker = SourceModule(get_error_vector_ker_function)
        get_error_ker = SourceModule(get_error_ker_function)

        self.get_error_vector = get_error_vector_ker.get_function("get_error_vector")
        self.get_error = get_error_ker.get_function("get_error")

In [52]:
## upper boundary: 5.8, downer boundary: 0.0

## destination
x_des = np.array([0,0,0,0,0,0])

## initial point
x_0 = np.array([100,0,-1500,-10,0,80])

## constraints
upper_boundary = 5.8
downer_boundary = 0.0

MECS = MinimumEnergyControlSolver(x_des, x_0, upper_boundary, downer_boundary)

In [53]:
MECS.solve()

MECS.MEC.u[:12]

array([[-0.3841809 ],
       [ 0.        ],
       [ 5.787262  ],
       [-0.38417292],
       [ 0.        ],
       [ 5.787263  ],
       [-0.38416386],
       [ 0.        ],
       [ 5.787264  ],
       [-0.38415447],
       [ 0.        ],
       [ 5.787265  ]], dtype=float32)

## Need to be tested

In [39]:
MECS.kernel_function()

In [40]:
MECS.evaluate(0)

In [41]:
MECS.error_vector[:6]

array([[   77.57094 ],
       [    0.      ],
       [-1166.0513  ],
       [  -10.648594],
       [    0.      ],
       [   85.85937 ]], dtype=float32)

In [42]:
G = MECS.MEC.G.get().reshape(150,6).T
u = MECS.MEC.u.get()
C = MECS.MEC.C.get()

In [43]:
G[:,:3]

array([[2.45125, 0.     , 0.     ],
       [0.     , 2.45125, 0.     ],
       [0.     , 0.     , 2.45125],
       [0.05   , 0.     , 0.     ],
       [0.     , 0.05   , 0.     ],
       [0.     , 0.     , 0.05   ]], dtype=float32)

In [44]:
error_G = np.dot(G, u) - C
print(error_G)

[[   77.57094 ]
 [    0.      ]
 [-1166.0514  ]
 [  -10.648594]
 [    0.      ]
 [   85.859375]]


In [45]:
MECS.error[0]

array([1172.0621,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    , 

In [46]:
rho_matrix = MECS.MEC.rho_matrix.get().reshape(150,150)
error_rho = np.dot(rho_matrix, u)

In [47]:
error = np.vstack((error_G, error_rho))
np.linalg.norm(error)

1172.0641

## Success!!!

In [54]:
MECS.error

array([[1495.1246, 1464.9031, 1420.0154, 1360.9907, 1288.5303, 1227.7255,
        1199.6062, 1185.2783, 1177.2913, 1172.5254, 1169.5496, 1167.6082,
        1166.2976, 1165.4529, 1164.911 , 1164.5176, 1164.3387, 1164.1848,
        1164.24  , 1164.3257, 1164.4106, 1164.5217, 1164.6606, 1164.8667,
        1165.0952, 1165.2686, 1165.4365, 1165.6218, 1165.8242, 1166.0298,
        1166.1436, 1166.2684, 1166.4047, 1166.5525, 1166.7117, 1166.8821,
        1167.0638, 1167.1965, 1167.2635, 1167.3368, 1167.4163, 1167.5017,
        1167.5933, 1167.6909, 1167.7948, 1167.9045, 1168.0206, 1168.1428,
        1168.271 , 1168.4054, 1168.5458, 1168.6921, 1168.8435, 1168.8645,
        1168.8877, 1168.9127, 1168.9402, 1168.9694, 1169.0007, 1169.0343,
        1169.07  , 1169.1075, 1169.1472, 1169.1892, 1169.2329, 1169.2789,
        1169.327 , 1169.3772, 1169.4293, 1169.4836, 1169.5399, 1169.5984,
        1169.6587, 1169.7212, 1169.7859, 1169.8525, 1169.921 , 1169.992 ,
        1170.0647, 1170.1395, 1170.216

In [57]:
MECS.error[0,0] < MECS.error[0,1]

array(0., dtype=float32)

In [58]:
MECS.error[0,0]

array(1495.1246, dtype=float32)

In [59]:
MECS.error[0,1]

array(1464.9031, dtype=float32)